In [47]:
import warnings
warnings.filterwarnings('ignore')#忽略一些警告
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn import tree

In [48]:
df = pd.read_csv('train.csv')
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [49]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


数据解释：Survived（0表示死亡，1表示幸存）：

PassengerId ：乘客ID
Pclass ：乘客等级(1/2/3等舱位)
Name ： 乘客姓名
Sex ： 性别
Age ： 年龄
SibSp ： 堂兄弟/妹个数
Parch ： 父母与小孩个数
Ticket ： 船票信息
Fare =：票价
Cabin ： 客舱
Embarked ： 登船港口
一共891条数据，年龄和客舱数据有缺失

In [50]:
X = df[['Pclass','Age','Sex']]
Y = df['Survived']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
Pclass    891 non-null int64
Age       714 non-null float64
Sex       891 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB


In [51]:
 #借由上面的输出，我们设计如下几个数据处理的任务：
# 1) age这个数据列，只有633个，需要补完。
# 2) sex 与 pclass两个数据列的值都是类别型的，需要转化为数值特征，用0/1代替。
# 首先我们补充age里的数据，使用平均数或者中位数都是对模型偏离造成最小影响的策略。


In [52]:
print(df.Age.mean())
print(df.Age.median()) 
print(df.Age.mode()) 

29.69911764705882
28.0
0    24.0
dtype: float64


中位数，众数，平均数都是29.69，就用它填充缺失值

In [53]:
df.Age.fillna(df.Age.mean(),inplace=True)

由于从df.head()中也可以看到客舱数据是类似于C85，C123等数据，这个数据本身对预测船上的人是否可以逃生应该是有帮助的，比如这个客舱位置是否靠近逃生通道，与撞击冰山的位置的远近等等，但由于没有相关的领域知识，而且数据缺失严重，考虑不把这个数据作为特征。年龄数据可以补充缺失值，一般可以用平均值来进行填充，我们先看一下有年龄数据的这些人的年龄平均值是多少。

基本的数据处理完毕后，就进入到特征选择环节了，这一块的话主要是会基于domain knowledge去选择一些特征，对于能否最后幸存，像姓名之类的一般无关。总体考虑下来像年龄，性别决定了个人的体力好坏，客舱级别也是一定的逃生的参考因素，以及是否有兄弟小孩也很有可能影响一个人是否能幸存，故而选取Pclass，Sex，Age，SibSp，Parch五个特征。当然这一块就如前所说通过领域知识和背景知识还可以创造出更多的特征，但这里也不是很熟练该如何创造特征，所以就先使用这五个特征。其中Sex给的是male和female由于是分类变量，算法识别不了字符串，还需要处理，将其变为0,1这样的数值,可以利用sklearn中的preprocessing

In [54]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['Sex'])
#用离散值转化标签值
df['Sex'] = le.transform(df['Sex'])
print(df.Sex)
#男1  女0

0      1
1      0
2      0
3      0
4      1
5      1
6      1
7      1
8      0
9      0
10     0
11     0
12     1
13     1
14     0
15     0
16     1
17     1
18     0
19     0
20     1
21     1
22     0
23     1
24     0
25     0
26     1
27     1
28     0
29     1
      ..
861    1
862    0
863    0
864    1
865    0
866    0
867    1
868    1
869    1
870    1
871    0
872    1
873    1
874    0
875    0
876    1
877    1
878    1
879    0
880    0
881    1
882    0
883    1
884    1
885    0
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64


In [55]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state = 33)

建模

In [56]:
features = ['Pclass','Sex','Age','SibSp','Parch']
X = df[features]
y = df['Survived']
dt = tree.DecisionTreeClassifier()
score = cross_val_score(dt,X,y,cv=5,scoring='accuracy')
import numpy as np
print(np.mean(score))

0.8081456507175897


In [57]:
#准确度0.8047747223460495

In [59]:
dt.fit(X,y)
from sklearn.tree import export_graphviz   #通过graphviz绘制决策树
with open('F:\Machine Learning\coding\Titanic\Titanic.dot','w') as f:
    f = export_graphviz(dt,feature_names=['Pclass','Sex','Age','SibSp','Parch'],out_file=f)

#export_graphviz第一个参数填入决策树的模型，feature_names填入参与的特征名，out_file即指定输出文件

这时候运行完后就能产生一个.dot文件，这个我们打不开，需要去网上下载graphviz后安装，安装完后打开将其bin目录添加到电脑的环境变量中，通过 dot -Tpng 文件名 -o 输出文件名的形式进行转化，当然也可以dot -Tpdf的形式，运行后我们就会在目录下看到Titanic.png了